# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [50]:
# TODOS: 
# 1) import any other libraries you might need
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
# 2) instantiate a Spark session 
spark = SparkSession\
    .builder\
    .appName("Wrangling Data")\
    .getOrCreate()

In [3]:
# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = "data/sparkify_log_small.json"
df = spark.read.json(path)

In [4]:
df.count()

10000

In [5]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
df.select("page").show(5)

+--------+
|    page|
+--------+
|NextSong|
|NextSong|
|NextSong|
|NextSong|
|    Home|
+--------+
only showing top 5 rows



In [7]:
# 4) write code to answer the quiz questions 

# Question 1

Which page did user id "" (empty string) NOT visit?

In [21]:
# TODO: write your code to answer question 1
empty_records = df.filter(df.userId == "")\
                    .select(col('page').alias("blank_pages")) \
                    .dropDuplicates()
# print(empty_records.count())

# get list of possible pages that could be visited
all_pages = df.select('page').dropDuplicates()
# print(all_pages.collect())
# print(all_pages.show())

# find values in all_pages that are not in blank_pages
for row in set(all_pages.collect()) - set(empty_records.collect()):
#     print(row)
    print(row.page)

Logout
Upgrade
Save Settings
Error
Downgrade
NextSong
Submit Upgrade
Submit Downgrade
Settings


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [23]:
# TODO: use this space to explore the behavior of the user with an empty string

# Perhaps it represents users who have not signed up yet or 
#     who are signed out and are about to log in.


# Question 3

How many female users do we have in the data set?

In [32]:
# TODO: write your code to answer question 3

df.filter(df.gender == 'F').select('userId', 'gender').dropDuplicates().count()
#     .select('userId','gender')

462

In [30]:
df.filter(df.gender == 'F') \
    .select('userId', 'gender') \
    .dropDuplicates() \
    .count()

462

# Question 4

How many songs were played from the most played artist?

In [40]:
# TODO: write your code to answer question 4

df.filter(df.page == 'NextSong')\
    .select('artist')\
    .groupBy('artist')\
    .agg({"artist":'count'})\
    .withColumnRenamed('count(artist)', 'Artistcount')\
    .sort(desc('Artistcount'))\
    .show(1)

+--------+-----------+
|  artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [41]:
# TODO: write your code to answer question 5
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [55]:
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))
# print(cusum.count())
cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

